<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_pseudo_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AF_pseudo_diffusion + proteinMPNN
Hacking AlphaFold to be a diffusion model (for backbone generation). At each step add logits from proteinMPNN.


**WARNING**: This notebook is experimental, not validated or tested.

In [1]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML
from google.colab import files
import numpy as np

CPU times: user 1.28 s, sys: 378 ms, total: 1.66 s
Wall time: 1min 56s


In [41]:
LENGTH = 100
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", use_templates=True)
af_model.prep_inputs(length=LENGTH)
mpnn_model = mk_mpnn_model()

In [42]:
af_model.restart(mode="gumbel")
L = af_model._len
af_model._inputs["batch"] = {"aatype":np.zeros(L).astype(int),
                             "all_atom_mask":np.zeros((L,37)),
                             "all_atom_positions":np.zeros((L,37,3))}

STEPS = 100 # might be overkill
for k in range(STEPS):
  # add noise
  noise_weight = 1 - k/STEPS
  af_model._inputs["batch"]["all_atom_positions"] += np.random.normal(size=(L,37,3)) * noise_weight

  # denoise
  aux = af_model.predict(return_aux=True, verbose=False)
  af_model._inputs["batch"]["aatype"] = af_model._params["seq"].argmax(-1)[0]
  af_model._inputs["batch"]["all_atom_mask"][:,:4] = 1
  af_model._inputs["batch"]["all_atom_positions"] = af_model.aux["atom_positions"]

  # add logits from proteinmpnn at each stage
  mpnn_model.get_af_inputs(af_model)
  mpnn_out = mpnn_model.score()
  aux["log"]["mpnn"] = mpnn_out["score"]
  af_model._params["seq"] = 0.9 * af_model._params["seq"] + 0.1 * mpnn_out["logits"][:,:20]

  # save results
  af_model._save_results(aux)
  af_model._k += 1

1 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.92 con 3.92 plddt 0.32 ptm 0.17 mpnn 2.81
2 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.90 con 3.90 plddt 0.33 ptm 0.17 mpnn 2.70
3 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.76 con 3.76 plddt 0.35 ptm 0.20 mpnn 2.69
4 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.58 con 3.58 plddt 0.40 ptm 0.26 mpnn 2.39
5 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.53 con 3.53 plddt 0.41 ptm 0.27 mpnn 2.42
6 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.50 con 3.50 plddt 0.43 ptm 0.30 mpnn 2.30
7 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.48 con 3.48 plddt 0.42 ptm 0.28 mpnn 2.07
8 models [0] recycles 0 hard 1 soft 0 temp 1 loss 3.09 con 3.09 plddt 0.55 ptm 0.40 mpnn 1.78
9 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.60 con 2.60 plddt 0.71 ptm 0.54 mpnn 1.63
10 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.49 con 2.49 plddt 0.75 ptm 0.56 mpnn 1.43
11 models [0] recycles 0 hard 1 soft 0 temp 1 loss 2.33 con

In [43]:
af_model.plot_pdb()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [44]:
af_model.save_pdb(f"tmp.pdb")
af_model.get_seqs()

['AAVVRLPDGWVPVANLDPRRPPGVPPVPVTPGYELDPAEVAALEAEYGPLDTSLFTNDPDQPLGCVKYDANGNVVAFKPLTVRDPETGELLLPVLRRPPP']

In [45]:
HTML(af_model.animate(dpi=100))